In [1]:
import pandas as pd
data = pd.read_csv('data.csv')
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
#Дадим столбцам корректные наименования

data = data.rename(columns = {'dob_years':'years', 'income_type':'employment_type', 'debt':'repayment_status',
                              'total_income':'monthly_income'})
data.columns

Index(['children', 'days_employed', 'years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'employment_type',
       'repayment_status', 'monthly_income', 'purpose'],
      dtype='object')

In [5]:
#Считаем количество пропусков
data.isnull().sum()

children               0
days_employed       2174
years                  0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
employment_type        0
repayment_status       0
monthly_income      2174
purpose                0
dtype: int64

Число пропусков в количестве рабочих дней и месячного дохода одинаковое. Скорее всего эти люди не получали доход, так как никогда не работали по разным причинам. Для дальнейшего анализа заполним пропуски нулями. Заполнять средним или медианным значением некорректно, так как это может повысить вероятность выдачи кредита неплатёжеспособному клиенту.

In [6]:
data['days_employed'] = data['days_employed'].fillna(0)
data['monthly_income'] = data['monthly_income'].fillna(0)
data.isnull().sum()

children            0
days_employed       0
years               0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
employment_type     0
repayment_status    0
monthly_income      0
purpose             0
dtype: int64

### Шаг 2.2 Проверка данных на аномалии и исправления.

Проверим каждый столбец отдельно на наличие аномалий

Начнём с детей


In [7]:
# Дети
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

Отрицательного количества детей "-1" быть не может. Значение 20 тоже выглядит странно. Нужно проверить общее количество таких записей.

In [8]:
data[data['children'] == -1]['children'].count()

47

In [9]:
# Заменим -1 на 1
data['children'] = data['children'].replace(-1, 1)

data[data['children'] == -1]['children'].count()

0

In [10]:
# Найдём количество записей в базе с 20 детьми
data[data['children'] == 20]['children'].count()

76

In [11]:
# Сгруппируем по возрасту, чтобы проверить корректность данных.
data[data['children'] == 20].groupby('years')['years'].count()

years
0     1
21    1
23    1
24    1
25    1
26    1
27    2
29    2
30    3
31    2
32    2
33    2
34    3
35    2
36    2
37    4
38    1
39    1
40    4
41    2
42    3
43    2
44    2
45    3
46    3
48    1
49    3
50    3
51    1
52    1
53    1
54    1
55    1
56    5
57    1
59    2
60    1
61    1
62    1
64    1
69    1
Name: years, dtype: int64

In [12]:
data['children'].value_counts()

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

Довольно много, но при этом равномерно распеределено по разным возрастам. Хотя логично было бы предположить, что такое число детей будет у более старших людей. К тому же странно, что семей с 4 и 5 детьми меньше, чем с 20. Скорее всего это опечатки при внесении данных в базу.

Заменим все значения 20 на 2 и проверим, что всё заменилось корректно:

In [13]:
data['children'] = data['children'].replace(20, 2)
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

**Теперь трудовой стаж в днях**

In [14]:
data['days_employed'].describe()

count     21525.000000
mean      56678.874622
std      134870.763085
min      -18388.949901
25%       -2518.168900
50%        -982.531720
75%           0.000000
max      401755.400475
Name: days_employed, dtype: float64

Во-первых отрицательных значений быть не может. Это явно ошибка при внесении данных. Надо изменить знак в таких ячейках. Это можно сделать с помощью функции возвращающей модуль числа.

In [15]:
data['days_employed'] = data['days_employed'].abs()
data['days_employed'].describe()

count     21525.000000
mean      60156.419005
std      133355.929525
min           0.000000
25%         610.652074
50%        1808.053434
75%        4779.587738
max      401755.400475
Name: days_employed, dtype: float64

Максимальное число неправдоподобно большое. Можно найти предельно возможное значение для данной ячейки. Для этого берём самого старшего клиента в нашей базе (максимальный возраст), предполагаем, что он работал с 16 лет (теоретически возможно) и считаем количество отработанных дней:

In [16]:
max_days_of_employment = (data['years'].max() - 16) * 365
max_days_of_employment

21535

Всё, что больше этого числа, скорее всего аномалия. Как обрабатывать такие значения в проекте не понятно. Посчитаем долю таких значений от общего числа.

In [17]:
data[data['days_employed'] > max_days_of_employment]['days_employed'].count() / 21525 * 100

16.004645760743323

Переведём дни стажа в года. Для этого добавим в таблицу новый столбец и приведём его к целочисленой форме

In [18]:
data['years_employed'] = data['days_employed'] / 365

data.head(10)

,children,days_employed,years,education,education_id,family_status,family_status_id,gender,employment_type,repayment_status,monthly_income,purpose,years_employed
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,23.116912
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,11.026860
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,15.406637
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,11.300677
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,932.235814
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,2.537495
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,7.888225
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,0.418574
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,18.985932
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,5.996593


Даже в этой небольшой части таблицы видим, что есть аномалии в виде невозможно большого стажа работы. Люди столько не живут.

**Кстати о возрасте, пора рассмотреть и этот столбец.**

In [19]:
data['years'].describe()

count    21525.000000
mean        43.293380
std         12.574584
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: years, dtype: float64

В целом правдоподобно, но есть нулевые значения. Возможно при заполнении данных кредитный менеджер ничего не внёс или ошибся. Посчитаем число таких записей.

In [20]:
data[data['years'] == 0]['years'].count()

101

Посмотрим на распределение по другим возрастам

In [21]:
data.groupby('years')['years'].count()

years
0     101
19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    597
43    513
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      8
74      6
75      1
Name: years, dtype: int64

И правда, очень похоже, что нулевые данные были внесены по ошибке. Заменим их на медианное значение.

In [22]:
data.loc[data['years'] == 0,'years'] = data['years'].median()

data.head(10)

,children,days_employed,years,education,education_id,family_status,family_status_id,gender,employment_type,repayment_status,monthly_income,purpose,years_employed
0,1,8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,23.116912
1,1,4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,11.026860
2,0,5623.422610,33.0,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,15.406637
3,3,4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,11.300677
4,0,340266.072047,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,932.235814
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,541.832241,51.0,высшее,0,женат / замужем,0,F,сотрудник,0,94187.878082,автомобиль,1.484472
96,0,0.000000,44.0,СРЕДНЕЕ,1,женат / замужем,0,F,сотрудник,0,0.000000,покупка жилой недвижимости,0.000000
97,0,0.000000,47.0,высшее,0,женат / замужем,0,F,сотрудник,0,0.000000,профильное образование,0.000000
98,0,364906.205736,54.0,высшее,0,женат / замужем,0,F,пенсионер,0,199707.298524,покупка жилья для сдачи,999.743029


**Образование**

In [23]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Всего типов образования - 5. Однако в списке у нас 15 уникальных значений. Приведём их все к единому формату - в нижнем регистре.

In [24]:
data['education'] = data['education'].str.lower()
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Отлично. Мы получили пять уникальных значений, как это и указано в столбце с идентификаторами образования.

Проведём аналогичную операцию и для столбца семейным положением.

In [25]:
data['family_status'] = data['family_status'].str.lower()
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

Проверим пол


In [26]:
data['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

Непонятное значение "XNA". Узнаем как часто оно встречается

In [27]:
data[data['gender']=='XNA'].count()

children            1
days_employed       1
years               1
education           1
education_id        1
family_status       1
family_status_id    1
gender              1
employment_type     1
repayment_status    1
monthly_income      1
purpose             1
years_employed      1
dtype: int64

Всего одно. Возможно какая-то ошибка в базе.

Проверим статусы.

In [28]:
data['employment_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

Внешне всё в порядке. Правка данных не требуется.


### Шаг 2.3. Изменение типов данных.

Среди имеющихся данных нет смысла в хранении чисел с плавающей точкой в столбцах со стажем и месячным доходом. Для принятия решения достаточно целого значения. Поэтому тип соответствующих данных можно заменить.

In [29]:
data['years_employed'] = data['years_employed'].astype(int)
data['monthly_income'] = data['monthly_income'].astype(int)

data.head(10)

,children,days_employed,years,education,education_id,family_status,family_status_id,gender,employment_type,repayment_status,monthly_income,purpose,years_employed
0,1,8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23
1,1,4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11
2,0,5623.422610,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15
3,3,4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11
4,0,340266.072047,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,932
5,0,926.185831,27.0,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,2
6,0,2879.202052,43.0,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,7
7,0,152.779569,50.0,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,0
8,2,6929.865299,35.0,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,18
9,0,2188.756445,41.0,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,5


### Шаг 2.4. Удаление дубликатов.

Определим число дубликатов в таблице

In [30]:
data.duplicated().sum()

72

Немного для такого объёма данных. Возможно причина их возникновения в программном обеспечении, отвечающем за ввод данных. Удалим имеющиеся дубликаты.

In [31]:
data = data.drop_duplicates().reset_index(drop=True)

In [33]:
# Проверим повторно, вдруг что-то пошло не так
data.duplicated().sum()

0

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Поскольку в таблице некоторые данные дублируются текстовой формой и идентификатором, можно вынести дубликаты в отдельные таблицы "словари"


In [34]:
data_education_id = data[['education','education_id']]
data_education_id = data_education_id.drop_duplicates().reset_index(drop=True)
data_education_id.head(10)

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [35]:
data_family_status_id = data[['family_status','family_status_id']]
data_family_status_id = data_family_status_id.drop_duplicates().reset_index(drop=True)
data_family_status_id.head(10)

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


Для удобства чтения таблицы удалим лишние столбцы: стаж в днях (оставим в годах), идентификаторы образования и семейного положения.

In [36]:
del data['days_employed']
del data['education_id']
del data['family_status_id']
data.head(10)

,children,years,education,family_status,gender,employment_type,repayment_status,monthly_income,purpose,years_employed
0,1,42.0,высшее,женат / замужем,F,сотрудник,0,253875,покупка жилья,23
1,1,36.0,среднее,женат / замужем,F,сотрудник,0,112080,приобретение автомобиля,11
2,0,33.0,среднее,женат / замужем,M,сотрудник,0,145885,покупка жилья,15
3,3,32.0,среднее,женат / замужем,M,сотрудник,0,267628,дополнительное образование,11
4,0,53.0,среднее,гражданский брак,F,пенсионер,0,158616,сыграть свадьбу,932
5,0,27.0,высшее,гражданский брак,M,компаньон,0,255763,покупка жилья,2
6,0,43.0,высшее,женат / замужем,F,компаньон,0,240525,операции с жильем,7
7,0,50.0,среднее,женат / замужем,M,сотрудник,0,135823,образование,0
8,2,35.0,высшее,гражданский брак,F,сотрудник,0,95856,на проведение свадьбы,18
9,0,41.0,среднее,женат / замужем,M,сотрудник,0,144425,покупка жилья для семьи,5


Теперь таблица стало гораздо удобнее для чтения.


### Шаг 2.6. Категоризация дохода.

Введём функцию для определения категорий дохода

In [37]:
def income_category(row):
    monthly_income = row['monthly_income']
    if monthly_income <= 30000:
        return 'E'
    elif monthly_income <= 50000:
        return 'D'
    elif monthly_income <= 200000:
        return 'C'
    elif monthly_income <= 1000000:
        return 'B'
    else:
        return 'A'

Введём новый столбец с категорией дохода клиента в таблицу и заполним его с помощью функции выше.

In [38]:
data['income_category'] = data.apply(income_category, axis=1)
data.head(10)

,children,years,education,family_status,gender,employment_type,repayment_status,monthly_income,purpose,years_employed,income_category
0,1,42.0,высшее,женат / замужем,F,сотрудник,0,253875,покупка жилья,23,B
1,1,36.0,среднее,женат / замужем,F,сотрудник,0,112080,приобретение автомобиля,11,C
2,0,33.0,среднее,женат / замужем,M,сотрудник,0,145885,покупка жилья,15,C
3,3,32.0,среднее,женат / замужем,M,сотрудник,0,267628,дополнительное образование,11,B
4,0,53.0,среднее,гражданский брак,F,пенсионер,0,158616,сыграть свадьбу,932,C
5,0,27.0,высшее,гражданский брак,M,компаньон,0,255763,покупка жилья,2,B
6,0,43.0,высшее,женат / замужем,F,компаньон,0,240525,операции с жильем,7,B
7,0,50.0,среднее,женат / замужем,M,сотрудник,0,135823,образование,0,C
8,2,35.0,высшее,гражданский брак,F,сотрудник,0,95856,на проведение свадьбы,18,C
9,0,41.0,среднее,женат / замужем,M,сотрудник,0,144425,покупка жилья для семьи,5,C


### Шаг 2.7. Категоризация целей кредита.

Посмотрим сколько уникальных целей для кредита и какие

In [39]:
unique_purpose = data['purpose'].unique().tolist()
print(len(unique_purpose))
unique_purpose

38


['покупка жилья',
 'приобретение автомобиля',
 'дополнительное образование',
 'сыграть свадьбу',
 'операции с жильем',
 'образование',
 'на проведение свадьбы',
 'покупка жилья для семьи',
 'покупка недвижимости',
 'покупка коммерческой недвижимости',
 'покупка жилой недвижимости',
 'строительство собственной недвижимости',
 'недвижимость',
 'строительство недвижимости',
 'на покупку подержанного автомобиля',
 'на покупку своего автомобиля',
 'операции с коммерческой недвижимостью',
 'строительство жилой недвижимости',
 'жилье',
 'операции со своей недвижимостью',
 'автомобили',
 'заняться образованием',
 'сделка с подержанным автомобилем',
 'получение образования',
 'автомобиль',
 'свадьба',
 'получение дополнительного образования',
 'покупка своего жилья',
 'операции с недвижимостью',
 'получение высшего образования',
 'свой автомобиль',
 'сделка с автомобилем',
 'профильное образование',
 'высшее образование',
 'покупка жилья для сдачи',
 'на покупку автомобиля',
 'ремонт жилью',
 '

Итого 38 различных значений, которые можно сгруппировать в 4 группы:
- операции с недвижимостью
- операции с автомобилем
- образование
- свадьба

Определить категорию можно по вхождению в значение в столбце "**purpose**" ключевых строк соответственно:
- "недвиж", "жиль"
- "автом"
- "образов"
- "свадьб"

Сделаем это в отдельной функции:

In [40]:
def purpose_category(row):
    purpose = row['purpose']
    if 'недвиж' in purpose or 'жиль' in purpose:
        return 'операции с недвижимостью'
    elif 'автом' in purpose:
        return 'операции с автомобилем'
    elif 'образов' in purpose:
        return 'образование'
    elif 'свадьб' in purpose:
        return 'свадьба'

In [41]:
data['purpose_category'] = data.apply(purpose_category, axis=1)
data.head(10)

,children,years,education,family_status,gender,employment_type,repayment_status,monthly_income,purpose,years_employed,income_category,purpose_category
0,1,42.0,высшее,женат / замужем,F,сотрудник,0,253875,покупка жилья,23,B,операции с недвижимостью
1,1,36.0,среднее,женат / замужем,F,сотрудник,0,112080,приобретение автомобиля,11,C,операции с автомобилем
2,0,33.0,среднее,женат / замужем,M,сотрудник,0,145885,покупка жилья,15,C,операции с недвижимостью
3,3,32.0,среднее,женат / замужем,M,сотрудник,0,267628,дополнительное образование,11,B,образование
4,0,53.0,среднее,гражданский брак,F,пенсионер,0,158616,сыграть свадьбу,932,C,свадьба
5,0,27.0,высшее,гражданский брак,M,компаньон,0,255763,покупка жилья,2,B,операции с недвижимостью
6,0,43.0,высшее,женат / замужем,F,компаньон,0,240525,операции с жильем,7,B,операции с недвижимостью
7,0,50.0,среднее,женат / замужем,M,сотрудник,0,135823,образование,0,C,образование
8,2,35.0,высшее,гражданский брак,F,сотрудник,0,95856,на проведение свадьбы,18,C,свадьба
9,0,41.0,среднее,женат / замужем,M,сотрудник,0,144425,покупка жилья для семьи,5,C,операции с недвижимостью


### Ответы на вопросы.

#### Есть ли зависимость между количеством детей и возвратом кредита в срок?

Для ответа на этот вопрос нужно построить сводную таблицу и посчитать долю клиентов с задолженностями для каждого значения количества детей:

Сначала посмотрим общее распределение клиентов по количеству детей. Эти данные помогут нам проверить правильность заполнения сводной таблицы.

In [42]:
children_set_grouped = data.groupby('children')['repayment_status'].count()
children_set_grouped

children
0    14090
1     4855
2     2128
3      330
4       41
5        9
Name: repayment_status, dtype: int64

Теперь соберём данные в сводную таблицу и узнаем долю неплательщиков от числа клиентов с конкретным количеством детей:

In [43]:
children_table = data.pivot_table(index = ['children'], columns = ['repayment_status'], values = 'purpose', aggfunc='count')

children_table['ratio'] = children_table[1] / (children_table[0] + children_table [1]) * 100
children_table

repayment_status,0,1,ratio
children,,,
0,13027.0,1063.0,7.544358
1,4410.0,445.0,9.165808
2,1926.0,202.0,9.492481
3,303.0,27.0,8.181818
4,37.0,4.0,9.756098
5,9.0,NaN,NaN


Суммы чисел в столбцах "0" и "1" совпадают с ранее посчитанным распределением. Значит данные корректны.

##### Вывод 1:

Люди с детьми отдают приблизительно на одинаковом уровне независимо от количества детей.
Однако важно отметить, что среди клиентов без детей самый маленький процент неплательщиков.

#### Есть ли зависимость между семейным положением и возвратом кредита в срок?

Также собираем сводную таблицу, но теперь для семейного положения.

In [45]:
family_status_grouped = data.groupby('family_status')['repayment_status'].count()
family_status_grouped

family_status
в разводе                 1195
вдовец / вдова             959
гражданский брак          4150
женат / замужем          12339
не женат / не замужем     2810
Name: repayment_status, dtype: int64

In [46]:
family_status_table = data.pivot_table(index=['family_status'], columns=['repayment_status'], values='monthly_income', aggfunc='count')
family_status_table['ratio'] = family_status_table[1] / (family_status_table[0] + family_status_table[0]) * 100
family_status_table

repayment_status,0,1,ratio
family_status,,,
в разводе,1110,85,3.828829
вдовец / вдова,896,63,3.515625
гражданский брак,3762,388,5.156831
женат / замужем,11408,931,4.080470
не женат / не замужем,2536,274,5.402208


##### Вывод 2

Самый низкий процент среди разведенных и вдоцов/вдов. А самый большой у неженатых/незамужных и живущих гражданским браком.

Судя по тому, что минимальное и максимальное значения отличаются более, чем в полтора раза, это имеет статистически важное значение при принятии решения о выдаче кредита.

#### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [47]:
monthly_income_grouped = data.groupby('income_category')['repayment_status'].count()
monthly_income_grouped

income_category
A       25
B     5041
C    13913
D      350
E     2124
Name: repayment_status, dtype: int64

По этим данным видно - чем выше уровень дохода, тем меньше неплательщиков. Но нужно проверить, изменится ли картина, если посчитать не абсолютное число должников, а их долю от общего числе клиентов в соответствующей категории.

In [48]:
monthly_income_table = data.pivot_table(index=['income_category'], columns=['repayment_status'], values='education', aggfunc='count')
monthly_income_table['ratio'] = monthly_income_table[1] / (monthly_income_table[0] + monthly_income_table[0]) * 100
monthly_income_table

repayment_status,0,1,ratio
income_category,,,
A,23,2,4.347826
B,4685,356,3.799360
C,12723,1190,4.676570
D,329,21,3.191489
E,1952,172,4.405738


##### Вывод 3

Ожидаемо большой на фоне других групп процент должников у людей с очень маленькими доходами.
При это практически такой же процент у людей с очень большим доходом. Но таких клиентов очень мало, чтобы по такой выборке делать далеко идущие выводы о потенциальных клиентами с очень большими доходами. Для такого анализа нужно больше данных для анализа.

Большую долю долгов у людей со средним доходом можно объяснить психологическим эффектом. Перейдя в эту категорию из более низкой люди начинат бездумно тратить деньги, думая, что этот уровень дохода сохранится и дальше. А так бывает далеко не всегда. В итоге клиенты выходят на просрочку.

#### Как разные цели кредита влияют на его возврат в срок?

Снова собираем сводную таблицу по цели кредита и наличию задолженности

In [49]:
purpose_category_grouped = data.groupby('purpose_category')['repayment_status'].count()
purpose_category_grouped

purpose_category
образование                  4013
операции с автомобилем       4306
операции с недвижимостью    10811
свадьба                      2323
Name: repayment_status, dtype: int64

In [50]:
purpose_category_table = data.pivot_table(index=['purpose_category'], columns=['repayment_status'], values='education', aggfunc='count')
purpose_category_table['ratio'] = purpose_category_table[1] / (purpose_category_table[0] + purpose_category_table[0]) * 100
purpose_category_table

repayment_status,0,1,ratio
purpose_category,,,
образование,3643,370,5.078232
операции с автомобилем,3903,403,5.162695
операции с недвижимостью,10029,782,3.898694
свадьба,2137,186,4.351895


Реже всего долги возникают у тех, кто берёт кредит на операции с недвижимостью. Это логично, так как такие сделки редко совершаются спонтанно, а срок кредита как правило больше, чем по другим категориям. А значит людям заранее нужно рассчитывать свои силы. Опять же кредиты на операции с недвижимостью как правило должны быть одобрены службой безопасности банка, которая оценивает заёмщика. Что помогает отсекать ненадёжных потенциальных клиентов.

Хуже всего смотрятся кредиты на образование и операции с автомобилями. Такие кредиты обычны для людей молодого поколения которые ещё не имеют стабильного дохода для погашения кредита. Нельзя не учесть возможный характер импульсивного принятия решения о таком кредите.

## Общий вывод:

Зависимость объёма просрочки от количества детей, семейного положения, уровня дохода и целей кредитования хоть и небольшая, но есть.

Особое внимание стоит обращать на неженатых/незамужних, а также на живущих гражданским браком. Количество должников в этих группах сильнее всего отличается от смежных. Если такие клиенты желают взять кредит для покупки автомобиля или образование, это должно стать поводов для дополнительной проверки заёмщика (через службу безопасности банка либо с помощью запроса у клиента дополнительных документов, подтверждающих доход).